In [15]:
import os
from os.path import dirname, abspath
import pandas as pd
import valentine as valentine
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import DistributionBased
import pprint

In [16]:
PATH = "C:/Users/fende/Desktop/Dati"
print(PATH)

C:/Users/fende/Desktop/Dati


Load data using pandas

In [17]:
df1 = pd.read_csv(PATH + "/bing_covid-19_data.csv", sep = ",", low_memory = False)
df1 = df1.drop(df1[df1.iso3 != 'USA' ].index)
df1['iso_subdivision'] = df1['iso_subdivision'].str[3:]
df2 = pd.read_csv(PATH + "/covid_tracking.csv", sep = ",", low_memory = False)

Keeping only the common values between the 2 datasets

In [18]:
# presi una tupla a caso per ogni stato della colonna iso2 per evitare di prendere i primi record di ogni stato ed effettuare un random sampling
df1 = df1.groupby(["iso_subdivision"]).sample(1)
df2 = df2.groupby(["state"]).sample(1)

merged = pd.merge(df1, df2, how = "inner", left_on=['iso_subdivision'], right_on=['state'])
#col_names_df2 = list(df2.columns)

merged.rename(columns={"recovered_x": "recovered", "iso_subdivision_x": "iso_subdivision", "load_time_x": "load_time"}, inplace=True)

merged = merged[df1.columns]

merged.to_csv(PATH + "/filtered_bing_tracking.csv", index = False)
#df2.to_csv(PATH + "/examples/data/filtered_policy_tracker.csv", index = False)

Instantiate matcher and run

In [19]:
%%time
matcher = DistributionBased(0.12, 0.12)
matches = valentine_match(merged, df2, matcher)

CPU times: total: 7.72 s
Wall time: 8.93 s


Ground truth for metrics calculation

In [20]:
ground_truth = [
                ("iso_subdivision", "state"),
                ("confirmed", "positive"),
               ]

Metrics calculation

In [21]:
metrics = valentine_metrics.all_metrics(matches, ground_truth)

pp = pprint.PrettyPrinter(indent=4)
print("Found the following matches:")
pp.pprint(matches)

print("\nAccording to the ground truth:")
pp.pprint(ground_truth)

print("\nThese are the scores of the matcher:")
pp.pprint(metrics)

Found the following matches:
{   (('table_1', 'confirmed'), ('table_2', 'fips')): 0.9114600074174806,
    (('table_1', 'confirmed'), ('table_2', 'fips_code')): 0.931957788832548,
    (('table_1', 'confirmed'), ('table_2', 'in_icu_currently')): 0.9453015930754118,
    (('table_1', 'confirmed'), ('table_2', 'on_ventilator_currently')): 0.9548150332579705,
    (('table_1', 'confirmed_change'), ('table_2', 'death_increase')): 0.9559747337775429,
    (('table_1', 'deaths'), ('table_2', 'fips')): 0.9086128930486909,
    (('table_1', 'deaths'), ('table_2', 'fips_code')): 0.9547057228098725,
    (('table_1', 'deaths'), ('table_2', 'in_icu_currently')): 0.9319550387455944,
    (('table_1', 'deaths'), ('table_2', 'on_ventilator_currently')): 0.9572417537667979,
    (('table_1', 'iso2'), ('table_2', 'iso_country')): 1.0,
    (('table_1', 'iso_subdivision'), ('table_2', 'state')): 0.9937313778792312,
    (('table_1', 'load_time'), ('table_2', 'load_time')): 1.0,
    (('table_1', 'recovered_change'